In [ ]:
import cv2
import torch
import os
import mediapipe as mp

# Load YOLOv5 model for detecting objects (humans)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True)

# Initialize MediaPipe Pose for pose estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to detect objects (humans) in the video frame
def detect_objects(frame):
    results = model(frame)
    detections = []
    if len(results.xyxy[0]) > 0:
        for *xyxy, conf, cls in results.xyxy[0]:
            label = model.names[int(cls)]
            if label == 'person' and conf > 0.5:  # Confidence threshold to detect humans
                x1, y1, x2, y2 = map(int, xyxy)
                detections.append({"label": label, "x1": x1, "y1": y1, "x2": x2, "y2": y2, "confidence": conf})
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return frame, detections

# Function to track the motion of humans in the video and classify as alive or dead
def track_human_activity(all_detections):
    active_people = 0
    inactive_people = 0
    unique_persons = set()  # A set to track unique persons across frames
    person_last_positions = {}  # Stores the last known position of each person
    movement_threshold = 50  # Minimum movement distance to be considered "alive"
    inactivity_threshold = 10  # Number of consecutive frames with no movement to be considered "dead"

    # Analyze the movement of each person
    for frame_data in all_detections:
        detections = frame_data["detections"]

        for detection in detections:
            label = detection["label"]
            if label == "person":
                # Use the bounding box (x1, y1, x2, y2) as a unique identifier for a person
                person_id = (detection["x1"], detection["y1"], detection["x2"], detection["y2"])

                # Track unique persons across frames using the bounding box
                if person_id not in unique_persons:
                    unique_persons.add(person_id)  # Add person to the unique set

                track_id = len(unique_persons)  # Track ID is the number of unique persons

                x1, y1, x2, y2 = detection["x1"], detection["y1"], detection["x2"], detection["y2"]
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2

                # Calculate movement: Compare the person's center position with the previous frame's position
                if track_id in person_last_positions:
                    prev_x, prev_y = person_last_positions[track_id]
                    distance_moved = ((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2) ** 0.5

                    # If the person moved more than the threshold, consider them as "active"
                    if distance_moved > movement_threshold:
                        active_people += 1
                    else:
                        inactive_people += 1

                # Update the last known position of the person
                person_last_positions[track_id] = (center_x, center_y)

    return active_people, inactive_people, len(unique_persons)

# Function to generate a conclusion based on the activity (alive or dead)
def generate_activity_report(all_detections):
    active_people, inactive_people, total_people = track_human_activity(all_detections)

    conclusion = "\n--- Conclusion ---\n"
    conclusion += f"Total number of unique people detected: {total_people}\n"
    conclusion += f"Active people (alive): {active_people}\n"
    conclusion += f"Inactive people (dead): {inactive_people}\n"

    if active_people > inactive_people:
        conclusion += "Most of the people in the video appear to be alive, with some inactive individuals."
    else:
        conclusion += "The video shows a higher number of inactive individuals, suggesting some may be dead or unconscious."

    return conclusion

# Function to process the video and generate the human actions story with alive/dead classification
def generate_video_story_with_status(video_file):
    if not os.path.exists(video_file):
        print(f"Error: Video file '{video_file}' not found.")
        return

    cap = cv2.VideoCapture(video_file)
    all_detections = []
    frame_count = 0
    story_text = ""

    if cap.isOpened():
        while cap.isOpened():
            ret, frame = cap.read()
            if ret:
                # Detect objects and human (person) in the video
                frame, detections = detect_objects(frame)
                all_detections.append({"frame": frame_count, "detections": detections})

                # Display the frame (optional)
                cv2.imshow('Frame', frame)
                cv2.waitKey(1)

                # Stop on 'q' key press
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

                frame_count += 1
            else:
                break

        cap.release()
        cv2.destroyAllWindows()

        # Generate conclusion for the activity report (alive vs. dead)
        conclusion = generate_activity_report(all_detections)
        story_text += conclusion

        # Save the generated story to a text file
        story_file = r"C:\Users\Administrator\Downloads\New Text Document (2).txt"
        try:
            with open(story_file, "w") as f:
                f.write(story_text)
            print(f"Story saved to {story_file}")
        except Exception as e:
            print(f"Error saving story to file: {e}")

        return story_text
    else:
        print("Could not open video file.")
        return None

# Main function to execute the process
video_file = r"C:\Users\Administrator\Downloads\WhatsApp Video 2025-02-20 at 13.45.59.mp4"  # Replace with your video file path
generate_video_story_with_status(video_file)
